In [2]:
import os
import warnings
import tqdm
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [3]:
%load_ext autoreload
%autoreload 2
import socceraction.spadl as spadl
import socceraction.vaep.formula as vaepformula

In [4]:
X_test = pd.read_csv("./soccer_multiclass_data/X_test",index_col=0)
Y_test = pd.read_csv("./soccer_multiclass_data/Y_test",index_col=0)
catboost_probability = pd.read_csv('./result/catboost_final_result',index_col=0)

In [5]:
X_test

,type_id_a0,type_id_a1,type_id_a2,type_pass_a0,type_cross_a0,type_throw_in_a0,type_freekick_crossed_a0,type_freekick_short_a0,type_corner_crossed_a0,type_corner_short_a0,...,end_dist_to_goal_a0,end_angle_to_goal_a0,end_dist_to_goal_a1,end_angle_to_goal_a1,end_dist_to_goal_a2,end_angle_to_goal_a2,team_1,team_2,time_delta_1,time_delta_2
0,0,0,0,True,False,False,False,False,False,False,...,62.344111,0.040050,62.344111,0.040050,62.344111,0.040050,True,True,0.0,0.0
1,21,0,0,False,False,False,False,False,False,False,...,59.963744,0.041641,62.344111,0.040050,62.344111,0.040050,True,True,1.0,1.0
2,0,21,0,True,False,False,False,False,False,False,...,63.970052,0.229408,59.963744,0.041641,62.344111,0.040050,True,True,1.0,2.0
3,0,0,21,True,False,False,False,False,False,False,...,77.353726,0.247320,63.970052,0.229408,59.963744,0.041641,True,True,3.0,4.0
4,21,0,0,False,False,False,False,False,False,False,...,77.140580,0.245717,77.353726,0.247320,63.970052,0.229408,True,True,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82099,0,21,0,True,False,False,False,False,False,False,...,44.177657,0.549396,75.302099,0.417845,75.302099,0.417845,True,True,2.0,3.0
82100,8,0,21,False,False,False,False,False,False,False,...,67.452732,0.365382,71.166049,0.330111,47.354672,0.701388,False,False,2.0,4.0
82101,3,8,0,False,False,False,True,False,False,False,...,6.565622,0.105074,48.423868,0.520962,44.177657,0.549396,False,True,28.0,30.0
82102,21,3,8,False,False,False,False,False,False,False,...,98.061188,0.025458,98.472996,0.006993,67.452732,0.365382,False,True,4.0,32.0


In [6]:
Y_test

,label
0,0
1,0
2,0
3,0
4,0
...,...
82099,0
82100,0
82101,0
82102,0


In [7]:
catboost_probability = pd.DataFrame(catboost_probability).rename(columns={'0': 'Not','1':'scores','2':'concedes'})
catboost_probability

,Not,scores,concedes
0,0.995310,0.003110,0.001579
1,0.993567,0.005205,0.001228
2,0.994960,0.004025,0.001015
3,0.996162,0.002569,0.001269
4,0.995850,0.002674,0.001476
...,...,...,...
82099,0.985394,0.013762,0.000844
82100,0.992078,0.000888,0.007034
82101,0.979576,0.007134,0.013290
82102,0.993870,0.003678,0.002452


In [8]:
from sklearn.metrics import roc_auc_score
import numpy as np
test_score = (np.array(Y_test) == 1).reshape(-1)
test_concede = (np.array(Y_test) == 2).reshape(-1)

print("catboost roc auc : " ,roc_auc_score(test_score, catboost_probability['scores']))
print("catboost roc auc : " ,roc_auc_score(test_concede, catboost_probability['concedes']))


catboost roc auc :  0.801841386477698
catboost roc auc :  0.8822706743867219


In [9]:
datafolder = "./data-fifa/La Liga"
#0은 train, 1은 valid, 2은 test

spadl2_h5 = os.path.join(datafolder, "spadl-statsbomb_test_competitions.h5")
features2_h5 = os.path.join(datafolder, "features_test.h5")
labels2_h5 = os.path.join(datafolder, "labels_test.h5")

predictions_h5 = os.path.join(datafolder, "catboost_predictions.h5")

In [10]:
test_games = pd.read_hdf(spadl2_h5, "games")
print("nb of games:", len(test_games))

nb of games: 35


In [11]:
Y_hat = catboost_probability[['scores','concedes']]
Y_hat

,scores,concedes
0,0.003110,0.001579
1,0.005205,0.001228
2,0.004025,0.001015
3,0.002569,0.001269
4,0.002674,0.001476
...,...,...
82099,0.013762,0.000844
82100,0.000888,0.007034
82101,0.007134,0.013290
82102,0.003678,0.002452


#### [78256, 78257, 78258, 782569]부분은 없어야 되는 값이므로
#### 그 후 값들이 한칸식 밀려나야함

In [12]:
remove_index = [78256, 78257, 78258, 78259]

In [13]:
append_index = [82104,82105,82106,82107]
for i in append_index:
    Y_hat.loc[i,'scores'] = 0
    Y_hat.loc[i,'concedes'] = 0

C:\Users\GunHee\AppData\Local\Temp\ipykernel_27316\1261452362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_hat.loc[i,'scores'] = 0


In [14]:
Y_hat

,scores,concedes
0,0.003110,0.001579
1,0.005205,0.001228
2,0.004025,0.001015
3,0.002569,0.001269
4,0.002674,0.001476
...,...,...
82103,0.003205,0.001885
82104,0.000000,0.000000
82105,0.000000,0.000000
82106,0.000000,0.000000


In [15]:
Y_hat = pd.concat([Y_hat.loc[:78255],Y_hat.loc[78256:].shift(4)]).reset_index().drop(columns=['index'])

In [16]:
Y_hat.loc[[78256,78257,78258,78259]]

,scores,concedes
78256,NaN,NaN
78257,NaN,NaN
78258,NaN,NaN
78259,NaN,NaN


In [17]:
Y_hat

,scores,concedes
0,0.003110,0.001579
1,0.005205,0.001228
2,0.004025,0.001015
3,0.002569,0.001269
4,0.002674,0.001476
...,...,...
82103,0.013762,0.000844
82104,0.000888,0.007034
82105,0.007134,0.013290
82106,0.003678,0.002452


In [18]:
# get rows with game id per action
A = []

for game_id in tqdm.tqdm(test_games.game_id, "Loading game ids"):
    Ai = pd.read_hdf(spadl2_h5, f"actions/game_{game_id}")
    A.append(Ai[["game_id"]])
A = pd.concat(A)
A = A.reset_index(drop=True)


Loading game ids: 100%|██████████| 35/35 [00:00<00:00, 109.66it/s]


In [19]:
A.loc[[78256,78257,78258,78259]]

,game_id
78256,3773689
78257,3773689
78258,3773689
78259,3773689


In [20]:
pd.concat([A, Y_hat],axis=1).loc[[78256,78257,78258,78259]]

,game_id,scores,concedes
78256,3773689,NaN,NaN
78257,3773689,NaN,NaN
78258,3773689,NaN,NaN
78259,3773689,NaN,NaN


In [21]:
# concatenate action game id rows with predictions and save per game
grouped_predictions = pd.concat([A, Y_hat], axis=1).groupby("game_id")
all_df = pd.DataFrame()
for k, df in tqdm.tqdm(grouped_predictions, desc="Saving predictions per game"):
    df = df.reset_index(drop=True)
    all_df = pd.concat([all_df,df]).reset_index(drop=True)
    df[Y_hat.columns].to_hdf(predictions_h5, f"game_{int(k)}")

Saving predictions per game: 100%|██████████| 35/35 [00:00<00:00, 72.24it/s]


In [22]:
df

,game_id,scores,concedes
0,3773695,0.003142,0.001908
1,3773695,0.004905,0.001486
2,3773695,0.004974,0.000696
3,3773695,0.003320,0.000866
4,3773695,0.002343,0.001172
...,...,...,...
2294,3773695,0.001958,0.005382
2295,3773695,0.007405,0.002775
2296,3773695,0.004650,0.001912
2297,3773695,0.004051,0.001338


#### game_id별로 집계하면 [78197, 78198, 78199, 78200]인덱스가 score,concede측정 불가능으로 설정해놓음

In [23]:
nan_index = [78197, 78198, 78199, 78200]

In [24]:
all_df[all_df['scores'].isnull()]

,game_id,scores,concedes
78197,3773689,NaN,NaN
78198,3773689,NaN,NaN
78199,3773689,NaN,NaN
78200,3773689,NaN,NaN


In [25]:
all_df[all_df['concedes'].isnull()]

,game_id,scores,concedes
78197,3773689,NaN,NaN
78198,3773689,NaN,NaN
78199,3773689,NaN,NaN
78200,3773689,NaN,NaN


In [26]:
# Configure file and folder names
datafolder = "./data-fifa/La Liga"
spadl_h5 = os.path.join(datafolder, "spadl-statsbomb_test_competitions.h5")
predictions_h5 = os.path.join(datafolder, "catboost_predictions.h5")

In [27]:
with pd.HDFStore(spadl_h5) as spadlstore:
    games = (
        spadlstore["games"]
        .merge(spadlstore["competitions"], how='left')
        .merge(spadlstore["teams"].add_prefix('home_'), how='left')
        .merge(spadlstore["teams"].add_prefix('away_'), how='left'))
    players = spadlstore["players"]
    teams = spadlstore["teams"]
print("nb of games:", len(games))

nb of games: 35


In [28]:

A = []
for game in tqdm.tqdm(list(games.itertuples()), desc="Rating actions"):
    if game.game_id==3773689:
        continue
    actions = pd.read_hdf(spadl_h5, f"actions/game_{game.game_id}")
    actions = (
        spadl.add_names(actions)
        .merge(players, how="left")
        .merge(teams, how="left")
        .sort_values(["game_id", "period_id", "action_id"])
        .reset_index(drop=True)
    )
    preds = pd.read_hdf(predictions_h5, f"game_{game.game_id}")
    
    values = vaepformula.value(actions, preds.scores, preds.concedes)
    A.append(pd.concat([actions, preds, values], axis=1))
    
A = pd.concat(A).sort_values(["game_id", "period_id", "time_seconds"]).reset_index(drop=True)
A.columns

Rating actions: 100%|██████████| 35/35 [00:01<00:00, 26.51it/s]


Index(['game_id', 'original_event_id', 'period_id', 'time_seconds', 'team_id',
       'player_id', 'start_x', 'start_y', 'end_x', 'end_y', 'type_id',
       'result_id', 'bodypart_id', 'action_id', 'type_name', 'result_name',
       'bodypart_name', 'player_name', 'nickname', 'team_name', 'scores',
       'concedes', 'offensive_value', 'defensive_value', 'vaep_value'],
      dtype='object')

In [29]:
A["count"] = 1

# Compute each player's number of actions and total VAEP values
playersR = (
    A[["player_id", "vaep_value", "offensive_value", "defensive_value", "count"]]
    .groupby(["player_id"])
    .sum()
    .reset_index()
)
# Add player names
playersR = playersR.merge(players[["player_id", "nickname", "player_name"]], how="left")
playersR["player_name"] = playersR[["nickname","player_name"]].apply(lambda x: x[0] if x[0] else x[1],axis=1)
# Show results
playersR = playersR[["player_id", "player_name", "vaep_value", "offensive_value", "defensive_value", "count"]]
playersR.sort_values("vaep_value", ascending=False)[:10]

,player_id,player_name,vaep_value,offensive_value,defensive_value,count
50,5503.0,Lionel Messi,31.015490,32.153340,-1.137850,5316
37,5203.0,Sergio Busquets,9.648888,9.338443,0.310445,4611
192,8118.0,Frenkie de Jong,5.538847,5.503440,0.035407,4956
38,5211.0,Jordi Alba,5.188876,4.907439,0.281437,5124
161,6826.0,Clément Lenglet,4.541694,5.077464,-0.535770,4222
370,43728.0,Óscar Mingueza,4.480322,3.627317,0.853005,2771
344,30756.0,Ansu Fati,3.053284,3.142048,-0.088764,502
249,20055.0,Marc-André ter Stegen,3.032779,2.082486,0.950292,1679
46,5477.0,Ousmane Dembélé,2.554726,2.996481,-0.441756,2112
352,32480.0,Ronald Araújo,1.903016,1.940468,-0.037452,1524


In [30]:
# Normalize for minutes played
pg = pd.read_hdf(spadl_h5, "player_games")
pg = pg[pg.game_id.isin(games.game_id)]
mp = pg[["player_id", "minutes_played"]].groupby("player_id").sum().reset_index()

stats = playersR.merge(mp)
stats = stats[stats.minutes_played > 180] # at least two full games played
stats["vaep_rating"] = stats.vaep_value * 90 / stats.minutes_played
stats["offensive_rating"] = stats.offensive_value * 90 / stats.minutes_played
stats["defensive_rating"] = stats.defensive_value * 90 / stats.minutes_played
stats.sort_values("vaep_rating",ascending=False)[:10]

,player_id,player_name,vaep_value,offensive_value,defensive_value,count,minutes_played,vaep_rating,offensive_rating,defensive_rating
50,5503.0,Lionel Messi,31.015490,32.153340,-1.137850,5316,3191,0.874771,0.906863,-0.032092
344,30756.0,Ansu Fati,3.053284,3.142048,-0.088764,502,446,0.616134,0.634046,-0.017912
81,6583.0,Carlos Soler,1.237529,1.335481,-0.097952,146,194,0.574111,0.619553,-0.045442
136,6746.0,Gabriel Paulista,0.845443,0.832497,0.012947,149,194,0.392216,0.386210,0.006006
164,6842.0,Iker Muniain,0.815721,0.838704,-0.022983,197,188,0.390505,0.401507,-0.011002
61,6297.0,Maxi Gómez,0.806241,0.806223,0.000018,73,194,0.374029,0.374021,0.000008
37,5203.0,Sergio Busquets,9.648888,9.338443,0.310445,4611,2407,0.360781,0.349173,0.011608
113,6665.0,Íñigo Martínez,0.711479,0.298333,0.413146,211,188,0.340602,0.142819,0.197783
222,11748.0,Unai Simón,0.504335,0.113247,0.391088,98,188,0.241437,0.054214,0.187223
370,43728.0,Óscar Mingueza,4.480322,3.627317,0.853005,2771,1754,0.229891,0.186122,0.043769


In [31]:
A[["concedes","offensive_value","defensive_value"]] = A[["concedes","offensive_value","defensive_value"]].round(decimals=4) 

In [32]:
for i,index in enumerate(a.index):
    text_name = f"({i+1}) {a.loc[index,'type_name']} \n {a.loc[index,'vaep_value']}"
    print(text_name)

NameError: name 'a' is not defined

In [41]:
temp = a[["time", "type_name", "player_name","scores", "concedes","vaep_value"]]
temp

,time,type_name,player_name,scores,concedes,vaep_value
34614,60m51s,dribble,Vinícius Júnior,0.018,0.001,-0.004
34615,60m51s,pass,Vinícius Júnior,0.019,0.001,0.001
34616,60m53s,dribble,Ferland Mendy,0.017,0.000,-0.002
34617,60m54s,pass,Ferland Mendy,0.073,0.000,0.057
34618,60m56s,dribble,Vinícius Júnior,0.066,0.000,-0.008
34619,60m56s,cross,Vinícius Júnior,0.015,0.000,-0.050


In [43]:
temp['type_name'].replace("_"," ")

34614    dribble
34615       pass
34616    dribble
34617       pass
34618    dribble
34619      cross
Name: type_name, dtype: object

In [52]:
for name in temp['player_name']:
    name_lst = name.split()
    first_name = name_lst[0][0] + ". "
    last_name = name_lst[-1]
    
    print(first_name + last_name)

V. Júnior
V. Júnior
F. Mendy
F. Mendy
V. Júnior
V. Júnior


In [63]:
sorted_A.game_id

Series([], Name: game_id, dtype: int64)

In [65]:
sorted_A = sorted_A[sorted_A.game_id==3773689]
sorted_A

,game_id,original_event_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,...,bodypart_name,player_name,nickname,team_name,scores,concedes,offensive_value,defensive_value,vaep_value,count


In [62]:
import matplotsoccer
import matplotlib.pyplot as plt
sorted_A = A.sort_values("vaep_value", ascending=False)
sorted_A = sorted_A[sorted_A.game_id==3773689]
#sorted_A = sorted_A[sorted_A.team_name == "Real Madrid"] # view only actions from Belgium
sorted_A = sorted_A[~sorted_A.type_name.str.contains("shot")] #eliminate shots
#sorted_A = sorted_A[(sorted_A['scores']<0.7) & (sorted_A['concedes']<0.7)]
def get_time(period_id,time_seconds):
    m = int((period_id-1)*45 + time_seconds // 60)
    s = time_seconds % 60
    if s == int(s):
        s = int(s)
    return f"{m}m{s}s"

for j in range(0, 20):
    row = list(sorted_A[j:j+1].itertuples())[0]
    i = row.Index
    a = A[i - 1: i+3].copy()
    
    a["player_name"] = a[["nickname", "player_name"]].apply(lambda x: x[0] if x[0] else x[1],axis=1)
    a['type_name'] = a['type_name'].replace("_"," ")
    
    for index, name in zip(a.index,a['player_name']):
        name_lst = name.split()
        first_name = name_lst[0][0] + ". "
        last_name = name_lst[-1]

        a.loc[index,'player_name'] = first_name + last_name
    
    g = list(games[games.game_id == a.game_id.values[0]].itertuples())[0]
    game_info = f"{g.game_date} {g.home_team_name} {g.home_score}-{g.away_score} {g.away_team_name}"
    minute = int((row.period_id-1)*45 + row.time_seconds // 60)
    print(f"{game_info} {minute}' {row.type_name} {row.player_name}")

    a["scores"] = a.scores.apply(lambda x : "%.4f" % x )
    a["concedes"] = a.concedes.apply(lambda x : "%.4f" % x )
    a["vaep_value"] = a.vaep_value.apply(lambda x : "%.3f" % x )
    a["time"] = a[["period_id","time_seconds"]].apply(lambda x: get_time(*x),axis=1)
    cols = ["time", "type_name", "player_name", "team_name", "result_name","scores", "concedes","offensive_value","defensive_value","vaep_value"]
    #cols = ["time", "type_name", "player_name","scores", "concedes","vaep_value"]
    matplotsoccer.actions(a[["start_x","start_y","end_x","end_y"]],
                a.type_name,
                team=a.team_name,
                result = a.result_name == "success",
                label=a[cols],
                labeltitle = cols,
                zoom=False)
    
    # for i,index in enumerate(a.index):
    #     text_name = f"({i+1}) {a.loc[index,'type_name']} \n {a.loc[index,'vaep_value']}"
    #     plt.text(a.loc[index,'start_x'],a.loc[index,'start_y'],text_name)

IndexError: list index out of range